#### KoAlpaca-Polyglat-12.8B 8bit 양자화 모델 추론 
```
1. excute: github: https://github.com/Beomi/KoAlpaca/tree/main/webui/app.py 
2. gqpt: github: https://github.com/qwopqwop200/GPTQ-for-KoAlpaca/blob/main/quant_with_alpaca.py
3. parameter data type: touch.float16
4. qutanization: 8 bit quant
5. processor: cpu
6. write type: pytorch bin
7. transformers: AutoModelForCausalLM, pipeline
```

In [1]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [2]:
# GPU 확인 함수
def isUsingGpu():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"GPU is available. Using GPU.{device}")
        return True
    else:
        device = torch.device("cpu")
        print(f"GPU is not available. Using CPU.{device}")
        return False

In [3]:
# model loaded porcessor
def is_model_on_gpu(model):
    return next(model.parameters()).is_cuda

In [4]:
if torch.cuda.is_available():
    torch.cuda.empty_cache() # 사용되지 않는 캐시된 메모리 해제
    print("GPU cache is emptied.")

GPU cache is emptied.


In [6]:
max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)}GB'
print(f"cuda max memory: {max_memory}")

cuda max memory: 14GB


In [7]:
# KoAlpaca-ployglot 12.8B 8BIT 양자화 모델
# 파일형식: PyTouch 모델 저장 방식 (확장자: bin)
# 파일크기: 9.5G, 3.1G 2개
model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bhf-8bit"

# GPU loading
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", 
    revision="8bit",
    load_in_8bit=True
    )

# CPU loading
# CPU 작동시에 에러가 발생함.
"""
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision="8bit",
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    device_map='cpu'
)
"""

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/data/bwllm/venv/llama/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_id,\n    revision="8bit",\n    load_in_8bit=True,\n    low_cpu_mem_usage=True,\n    device_map=\'cpu\'\n)\n'

In [8]:
# 로딩 모델 위치
next(model.parameters()).device

device(type='cuda', index=0)

In [9]:
# 전체 파라메터 확인
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_parameters}")

Total trainable parameters: 308060160


In [10]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

First parameter data type: torch.float16


In [11]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

The model is not quantized.


In [12]:
# 장치 확인   
using_cuda = isUsingGpu()
on_gpu_model = is_model_on_gpu(model)

print(f"GPU Using: {using_cuda}, Model GPU Loaded: {on_gpu_model}")
# GPU 가 사용가능하고, 모델이 CPU에 로딩되어 있다면
# 모델을 GPU 로 이동한다.
if(using_cuda == True and on_gpu_model == False):
    model.to('cuda')

GPU is available. Using GPU.cuda
GPU Using: True, Model GPU Loaded: True


In [14]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
          (dense): Linear8bitLt(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear8bitLt(in_features=20480, out_f

In [13]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

gpt_neox.embed_in.weight data type: torch.float16
gpt_neox.layers.0.input_layernorm.weight data type: torch.float16
gpt_neox.layers.0.input_layernorm.bias data type: torch.float16
gpt_neox.layers.0.post_attention_layernorm.weight data type: torch.float16
gpt_neox.layers.0.post_attention_layernorm.bias data type: torch.float16
gpt_neox.layers.0.attention.query_key_value.weight data type: torch.int8
gpt_neox.layers.0.attention.query_key_value.bias data type: torch.float16
gpt_neox.layers.0.attention.dense.weight data type: torch.int8
gpt_neox.layers.0.attention.dense.bias data type: torch.float16
gpt_neox.layers.0.mlp.dense_h_to_4h.weight data type: torch.int8
gpt_neox.layers.0.mlp.dense_h_to_4h.bias data type: torch.float16
gpt_neox.layers.0.mlp.dense_4h_to_h.weight data type: torch.int8
gpt_neox.layers.0.mlp.dense_4h_to_h.bias data type: torch.float16
gpt_neox.layers.1.input_layernorm.weight data type: torch.float16
gpt_neox.layers.1.input_layernorm.bias data type: torch.float16
gpt_ne

In [15]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [17]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [18]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘


In [19]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


!

### 맥락:
정보화 사업에 대한 감리의 절차는 정보화추진위원회의 심의를 거친 사업에 대하여 정보통신부장관이 감리를 지정하도록 되어 있습니다.그리고 감리자는 감리원을 지정하여 감리계획을 수립하고 감리를 실시하도록 되어 있습니다.
1. 감리절차 : 
                                                                            

2. 감리보고서 :
                                                                            


### 답변:정보화 시스템 감리는 정보통신부장관이 감리를 지정하면, 감리자는 감리계획을 수립하여 감리를 수행하게 됩니다. 일반적으로 정보화 시스템 구축 사업은 제안요청서(RFP)를 통해 사업이 발주되며, 제안서 평가, 사업자 선정, 협약 체결, 사업 추진, 감리 등의 과정을 거칩니다. 감리는 기술적인 사항뿐만 아니라, 사업의 목표와 범위, 예산의 적정성 등을 검토하여 사업 진행상의 문제점을 찾아내고 개선합니다. 

또한, 정보화 시스템 구축 사업의 감리는 정보통신부로부터 감리인 지정을 받아 정보화추진위원회의 심의를 거친 사업에 대해 정보통신부에 제출합니다. 이후, 정보통신부는 감리인을 지정하여 해당 사업에 대한 감리를 수행하게 합니다. 

감리보고서는 정보화추진위원회와 정보통신부가 협의하여 작성하며, 감리 과정에서 발견된 문제점과 개선이 필요한 부분 등을 기록합니다. 

 참고자료: 
- http://www.mic.go.kr/user.tdf?a=mic.subindex.MicSubIndexApp&c=3001⊂=1&catdepth=2&catdepth2=&menu_idx=2&menu_idx2=&layer=⊂=1&layer=⊃=1&mic=MI0050100&schType=&schText=&schStartDate=&schEndDate=&schCatchCount=&schCatchLength=&schApplyAntiAntiSubS